## <b><font color='darkblue'>Preface</font></b>
([article source](https://superfastpython.com/multiprocessing-in-python/)) <b><font size='3ptx'>Python Multiprocessing provides parallelism in Python with processes.</font></b>

The multiprocessing API uses process-based concurrency and is the preferred way to implement parallelism in Python. <b>With multiprocessing, we can use all CPU cores on one system, whilst avoiding [Global Interpreter Lock](https://realpython.com/python-gil/)</b>.

This notebook provides a detailed and comprehensive walkthrough of the [**Python Multiprocessing API**](https://docs.python.org/3/library/multiprocessing.html).

<a id='agenda'></a>
### <b><font color='darkgreen'>Table of Contents</font></b>
* <b><font size='3ptx'><a href='#sect1'>Python Processes</a></font></b>
* <b><font size='3ptx'><a href='#sect2'>Run a Function in a Process</a></font></b>
* <b><font size='3ptx'><a href='#sect3'>Extend the Process Class</a></font></b>
* <b><font size='3ptx'><a href='#sect4'>Process Start Methods</a></font></b>
* <b><font size='3ptx'><a href='#sect5'>Process Instance Attributes</a></font></b>
* <b><font size='3ptx'><a href='#sect6'>Configure Processes</a></font></b>
* <b><font size='3ptx'><a href='#sect7'>Main Process</a></font></b>
* <b><font size='3ptx'><a href='#sect8'>Process Utilities</a></font></b>
* <b><font size='3ptx'><a href='#sect9'>Process Mutex Lock</a></font></b>
* <b><font size='3ptx'><a href='#sect10'>Process Reentrant Lock</a></font></b>

In [2]:
import multiprocessing
from multiprocessing import Process
from time import sleep

<a id='sect1'></a>
## <b><font color='darkblue'>Python Processes</font></b>
<b><font size='3ptx'>So what are processes and why do we care?</font></b>
* <b><a href='#sect1_1'>What Are Processes</a></b>
* <b><a href='#sect1_2'>Thread vs Process</a></b>
* <b><a href='#sect1_3'>Life-Cycle of a Process</a></b>
* <b><a href='#sect1_4'>Child vs Parent Process</a></b>

<a id='sect1_1'></a>
### <b><font color='darkgreen'>What Are Processes</font></b>
<b><font size='3ptx'>A process refers to a computer program.</font></b>

<b>Every Python program is a process and has one default thread called the main thread used to execute your program instructions. Each process is, in fact, one instance of the Python interpreter that executes Python instructions</b> (Python byte-code), which is a slightly lower level than the code you type into your Python program.

Sometimes we may need to create new processes to run additional tasks concurrently. Python provides real system-level processes via the [**multiprocessing.Process**](https://docs.python.org/3/library/multiprocessing.html#the-process-class) class in the [**multiprocessing**](https://docs.python.org/3/library/multiprocessing.html#) module.

The underlying operating system controls how new processes are created. On some systems, that may require spawning a new process, and on others, it may require that the process is forked. <b>The operating-specific method used for creating new processes in Python is not something we need to worry about as it is managed by your installed Python interpreter</b>.

The code in new child processes may or may not be executed in parallel (<font color='brown'>at the same time</font>), even though the threads are executed concurrently.

There are a number of reasons for this, such as the underlying hardware may or may not support parallel execution (<font color='brown'>e.g. one vs multiple CPU cores</font>). This highlights the distinction between code that can run out of order (<font color='brown'>concurrent</font>) from the capability to execute simultaneously (<font color='brown'>parallel</font>).
* <b>Concurrent</b>: Code that can be executed out of order.* <b>
Paralle</b>l: Capability to execute code simultaneously

<b>Processes and threads are different.</b> Next, let’s consider the important differences between processes and threads..

<a id='sect1_2'></a>
### <b><font color='darkgreen'>Thread vs Process</font></b>
<b><font size='3ptx'>By definition, a process is an instance of a program running on a computer. And a thread is a unit of execution within a process.</font></b>

Each process is in fact one instance of the Python interpreter that executes Python instructions (Python byte-code), which is a slightly lower level than the code you type into your Python program.
> Process: The operating system’s spawned and controlled entity that encapsulates an executing application. A process has two main functions. The first is to act as the resource holder for the application, and the second is to execute the instructions of the application. <br/><br/>
> <b><a href='https://amzn.to/3J74TRr'>— PAGE 271, THE ART OF CONCURRENCY, 2009.</a></b>

<br/>

The operating-specific method used for creating new processes in Python is not something we need to worry about as it is managed by your installed Python interpreter. <b>A thread always exists within a process and represents the manner in which instructions or code is executed.</b>

<b>A process will have at least one thread, called the main thread. Any additional threads that we create within the process will belong to that process</b>. The Python process will terminate once all (<font color='brown'>non background threads</font>) are terminated.

* **Process**: An instance of the Python interpreter has at least one thread called the MainThread.
* **Thread**: A thread of execution within a Python process, such as the MainThread or a new thread

You can learn more about the differences between processes and threads in the tutorial "[**Thread vs Process in Python**](https://superfastpython.com/thread-vs-process)" . Next, let’s take a look at processes in Python...

<a id='sect1_3'></a>
### <b><font color='darkgreen'>Life-Cycle of a Process</font></b>
<b><font size='3ptx'>A process in Python is represented as an instance of the [**multiprocessing.Process**](https://docs.python.org/3/library/multiprocessing.html#the-process-class) class.</font></b>

Once a process is started, the Python runtime will interface with the underlying operating system and request that a new native process be created. The [**multiprocessing.Process**](https://docs.python.org/3/library/multiprocessing.html#the-process-class) instance then provides a Python-based reference to this underlying native process.

Each process follows the same life cycle. Understanding the stages of this life-cycle can help when getting started with concurrent programming in Python.

For example:
* The difference between creating and starting a process.
* The difference between run and start.
* The difference between blocked and terminated.

A Python process may progress through three steps of its life cycle: a new process, a running process, and a terminated process.

While running, the process may be executing code or may be blocked, waiting on something such as another process or an external resource. Although not all processes may block, it is optional based on the specific use case for the new process:

1. New Child Process.
2. Running Process.
3. Blocked Process (optional)
4. Terminated Process.

A new process is a process that has been constructed and configured by creating an instance of the [**multiprocessing.Process**](https://docs.python.org/3/library/multiprocessing.html#the-process-class) class.

<b>A new process can transition to a running process by calling the <font color='blue'>start()</font> method. This also creates and starts the main thread for the process that actually executes code in the process</b>. A running process may block in many ways if its main thread is blocked, such as reading or writing from a file or a socket or by waiting on a concurrency primitive such as a semaphore or a lock. After blocking, the process will run again.

Finally, a process may terminate once it has finished executing its code or by raising an error or exception.

A process cannot terminate until:
* All non-daemon threads have terminated, including the main thread.
* All non-daemon child processes have terminated, including the main process

You can learn more about the life-cycle of processes in the tutorial "[**Process Life-Cycle in Python**](https://superfastpython.com/process-life-cycle)". Next, let’s take a closer look at the difference between parent and child processes..

<a id='sect1_4'></a>
### <b><font color='darkgreen'>Child vs Parent Process</font></b>
<b><font size='3ptx'>Parent processes have one or more child processes, whereas child processes is created by a parent process.</font></b>

#### <b>Parent Process</b>
A parent process is a process that is capable of starting child processes.

Typically, we would not refer to a process as a parent process until it has created one or more child processes. This is the commonly accepted definition:
> <b>Parent Process:</b> Has one or more child processes. May have a parent process, e.g. may also be a child process.

<br/>

Recall, a process is an instance of a computer program. In Python, a process is an instance of the Python interpreter that executes Python code. <b>In Python, the first process created when we run our program is called the ‘MainProcess‘</b>. It is also a parent process and may be called the main parent process.

The main process will create the first child process or processes. A child process may also become a parent process if it in turn creates and starts a child process.

#### <b>Child Process</b>
A child process is a process that was created by another process.

A child process may also be called a subprocess, as it was created by another process rather than by the operating system directly. That being said, the creation and management of all processes is handled by the underlying operating system.
> <b>Child Process</b>: Has a parent process. May have its own child processes, e.g. may also be a parent.

<br/>

The process that creates a child process is called a parent process. A child process only ever has one parent process. There are three main techniques used to create a child process, referred to as process start methods.
> They are: fork, spawn, and fork server.

<br/>

Depending on the technique used to start the child, the child process may or may not inherit properties of the parent process. For example, a forked process may inherit a copy of the global variables from the parent process.

A child process may become orphaned if the parent process that created it is terminated. You can learn more about the differences between child and parent processes in the tutorial "[**Parent Process vs Child Process in Python**](https://superfastpython.com/parent-process-vs-child-process-in-python)".

Now that we are familiar with child and parent processes, let’s look at how to run code in new processes.

<a id='sect2'></a>
## <b><font color='darkblue'>Run a Function in a Process</font></b>
<b><font size='3ptx'>Python functions can be executed in a separate process using the [multiprocessing.Process](https://docs.python.org/3/library/multiprocessing.html#the-process-class) class.</font></b>
* <b><a href='#sect2_1'>How to Run a Function In a Process?</a></b>
* <b><a href='#sect2_2'>Example of Running a Function in a Process</a></b>
* <b><a href='#sect2_3'>Example of Running a Function in a Process With Arguments</a></b>

In this section we will look at a few examples of how to run functions in a child process.

<a id='sect2_1'></a>
### <b><font color='darkgreen'>How to Run a Function In a Process</font></b>
To run a function in another process:
* Create an instance of the [**multiprocessing.Process**](https://docs.python.org/3/library/multiprocessing.html#the-process-class) class.
* Specify the name of the function via the “target” argument.
* Call the <font color='blue'>start()</font> function.

First, we must create a new instance of the [**multiprocessing.Process**](https://docs.python.org/3/library/multiprocessing.html#the-process-class) class and specify the function we wish to execute in a new process via the “target” argument:

```python
...
# create a process
process = multiprocessing.Process(target=task)
```

<br/>

The function executed in another process may have arguments in which case they can be specified as a tuple and passed to the “args” argument of the multiprocessing.Process class constructor or as a dictionary to the “kwargs” argument:

```python
...
# create a process
process = multiprocessing.Process(target=task, args=(arg1, arg2))
```

<br/>

We can then start executing the process by calling the <font color='blue'>start()</font> function. The <font color='blue'>start()</font> function will return immediately and the operating system will execute the function in a separate process as soon as it is able:
```python
...
# run the new process
process.start()
```

<br/>

A new instance of the Python interpreter will be created and a new thread within the new process will be created to execute our target function. And that’s all there is to it.

We do not have control over when the process will execute precisely or which CPU core will execute it. Both of these are low-level responsibilities that are handled by the underlying operating system.

Next, let’s look at a worked example of executing a function in a new process.

<a id='sect2_2'></a>
### <b><font color='darkgreen'>Example of Running a Function in a Process</font></b>
<b><font size='3ptx'>First, we can define a custom function that will be executed in another process.</font></b>

We will define a simple function that blocks for a moment then prints a statement. The function can have any name we like, in this case we’ll name it “task“.

In [2]:
# a custom function that blocks for a moment
def task():
    # block for a moment
    sleep(1)
    # display a message
    print('This is from another process')

Next, we can create an instance of the [**multiprocessing.Process**](https://docs.python.org/3/library/multiprocessing.html#the-process-class) class and specify our function name as the “target” argument in the constructor.

In [5]:
# create a process
process = Process(target=task)

Once created we can run the process which will execute our custom function in a new native process, as soon as the operating system can:

```python
# run the process
process.start()
```

The <font color='blue'>start()</font> function does not block, meaning it returns immediately. We can explicitly wait for the new process to finish executing by calling the <font color='blue'>join()</font> function.

```python
...
# wait for the process to finish
print('Waiting for the process...')
process.join()
```

Tying this together, the complete example of executing a function in another process is listed below:

In [7]:
# run the process
process.start()

# wait for the process to finish
print('Waiting for the process...')
process.join()
print('Process is done!')

Waiting for the process...
This is from another process
Process is done!


Running the example first creates the [**multiprocessing.Process**](https://docs.python.org/3/library/multiprocessing.html#the-process-class) then calls the <font color='blue'>start()</font> function. <b>This does not start the process immediately, but instead allows the operating system to schedule the function to execute as soon as possible</b>.

At some point a new instance of the Python interpreter is created that has a new thread which will execute our target function.

You could use this script `test_subprocess_ex1.py` to learn more:
```shell
$ ./test_subprocess_ex1.py &
[1] 80327
Parent process id: 803279
Waiting for the process...
Subprocess PID: 803280
Current working directory: /usr/local/google/home/johnkclee/Gitrepos/python_101_crash_courses/article_collection/Multiprocessing_in_python
Touch file "STOP" to stop the task...
Subprocess PID: 803281
Current working directory: /usr/local/google/home/johnkclee/Gitrepos/python_101_crash_courses/article_collection/Multiprocessing_in_python
Touch file "STOP" to stop the 

$ pstree -p 803279
python(803279)─┬─python(803280)
               └─python(80328

$ touch STOP
Stopping
Stopping
Sub processes are all done!
Sleep 5s for main process to quit

$ pstree -p 803279
python(803279)...

Next, let’s look at how we might run a function with arguments in a new process.1)task...9
```

<a id='sect2_3'></a>
### <b><font color='darkgreen'>Example of Running a Function in a Process With Arguments</font></b> ([back](#sect2))
<b><font size='3ptx'>We can execute functions in another process that takes arguments.</font></b>

This can be demonstrated by first updating our <font color='blue'>task2()</font> function from the previous section to take two arguments, one for the time in seconds to block and the second for a message to display:

```python
# a custom function that blocks for a moment
def task2(sleep_time, message):
    # block for a moment
    sleep(sleep_time)
    # display a message
    print(message)
```

<br/>

Next, we can update the call to the [**multiprocessing.Process**](https://docs.python.org/3/library/multiprocessing.html#the-process-class) constructor to specify the two arguments in the order that our <font color='blue'>task2()</font> function expects them as a tuple via the “args” argument:


```python
...
# create a process
process = Process(target=task, args=(1.5, 'New message from another process'))
```

Tying this together, the complete example of executing a custom function that takes arguments in a separate process is listed below.
- `test_subprocess_ex2.py`:

```python
# example of running a function with arguments in another process
import os

from time import sleep
from multiprocessing import Process


# a custom function that blocks for a moment
def task2(sleep_time, message):
  print(f'Subprocess with PID: {os.getpid()}')
  # block for a moment
  sleep(sleep_time)
  # display a message
  print(message)
  print(f'Exit from {os.getpid()}')


# entry point
if __name__ == '__main__':
  print(f'Main process with PID: {os.getpid()}')

  # create a process
  process1 = Process(target=task2, args=(1.5, 'New message from another process'))
  process2 = Process(target=task2, args=(2, 'New message from another process'))

  # run the process
  process1.start()
  process2.start()

  # wait for the process to finish
  print('Waiting for the process...')
  process1.join()
  process2.join()

  print('Bye!')
```

You can learn more about running functions in new processes in this tutorial "[**Run a Function in a Child Process**](https://superfastpython.com/run-function-in-new-process/)".

Next let’s look at how we might run a function in a child process by extending th[**e multiprocessing.Proce**](https://docs.python.org/3/library/multiprocessing.html#the-process-class)ss class.

<a id='sect3'></a>
## <b><font color='darkblue'>Extend the Process Class</font></b> ([back](#agenda))
<b><font size='3ptx'>We can also execute functions in a child process by extending the [multiprocessing.Process](https://docs.python.org/3/library/multiprocessing.html#the-process-class) class and overriding the <font color='blue'>run()</font> function.</font></b>
* <b><a href='#sect3_1'>How to Extend the Process Class</a></b>
* <b><a href='#sect3_2'>Example of Extending the Process Class</a></b>
* <b><a href='#sect3_3'>Example of Extending the Process Class and Returning Values</a></b>

In this section we will look at some examples of extending the [**multiprocessing.Process**](https://docs.python.org/3/library/multiprocessing.html#the-process-class) class.

<a id='sect3_1'></a>
### <b><font color='darkgreen'>How to Extend the Process Class</font></b>
<b><font size='3ptx'>The [multiprocessing.Process](https://docs.python.org/3/library/multiprocessing.html#the-process-class) class can be extended to run code in another process.</font></b>

This can be achieved by first extending the class, just like any other Python class. For example:

```python
# custom process class
class CustomProcess(multiprocessing.Process):
    # ...
```

Then the <font color='blue'>run()</font> function of the [**multiprocessing.Process**](https://docs.python.org/3/library/multiprocessing.html#the-process-class) class must be overridden to contain the code that you wish to execute in another process:


```python
class CustomProcess(multiprocessing.Process):
  # ...

  def run(self):
    # ...    
```

You can also define additional functions in the class to split up the work you may need to complete in another process.

Next, let’s look at a worked example of extending the multiprocessing.Process class.

<a id='sect3_2'></a>
### <b><font color='darkgreen'>Example of Extending the Process Class</font></b> ([back](#sect3))
The complete example of executing code in another process by extending the [**multiprocessing.Process**](https://docs.python.org/3/library/multiprocessing.html#the-process-class) class is listed below:

- `test_subprocess_ex3.py`:

```python
import os

from time import sleep
from multiprocessing import Process


STOP_SIGNAL = os.path.join(os.getcwd(), 'STOP')


class MyProcess(Process):

  def __init__(self, name: str):
    super().__init__()
    self.name = name

  def run(self):
    print(f'Subprocess of "{self.name}" with pid: {os.getpid()}')
    print('Wait for STOP signal...')
    while not os.path.isfile(STOP_SIGNAL):
      sleep(1)

    print(f'Exit subprocess({os.getpid()})')


if __name__ == '__main__':
  print(f'Main process with pid: {os.getpid()}')

  # 1) Create processes
  my_process1 = MyProcess('John')
  my_process2 = MyProcess('Mary')

  # 2) Start the processes
  my_process1.start()
  my_process2.start()

  # 3) Join the processes
  my_process1.join()
  my_process2.join()

  os.remove(STOP_SIGNAL)

  print('Bye!')

```

You can learn more about extending the multiprocessing.Process class in this tutorial "[**How to Extend the Process Class in Python**](https://superfastpython.com/extend-process-class/)".

Next, let’s look at how we might return values from a child process.

<a id='sect3_3'></a>
### <b><font color='darkgreen'>Example of Extending the Process Class and Returning Values</font></b> ([back](#sect3))
<b><font size='3ptx'>Instance variable attributes can be shared between processes via the [multiprocessing.Value](https://docs.python.org/3/library/multiprocessing.html#multiprocessing.Value) and [multiprocessing.Array](https://docs.python.org/3/library/multiprocessing.html#multiprocessing.Array) classes.</font></b>

These classes explicitly define data attributes designed to be shared between processes in a process-safe manner. <b>Shared variables mean that changes made in one process are always propagated and made available to other processes.</b>

An instance of the [**multiprocessing.Value**](https://docs.python.org/3/library/multiprocessing.html#multiprocessing.Value) can be defined in the constructor of a custom class as a shared instance variable. The constructor of the [**multiprocessing.Value**](https://docs.python.org/3/library/multiprocessing.html#multiprocessing.Value) class requires that we specify the data type and an initial value.

The data type can be specified using ctype “type” or a typecode.

We can define an instance attribute as an instance of the [**multiprocessing.Value**](https://docs.python.org/3/library/multiprocessing.html#multiprocessing.Value) which will automatically and correctly be shared between processes. We can update the above example to use a [**multiprocessing.Value**](https://docs.python.org/3/library/multiprocessing.html#multiprocessing.Value) directly.

The complete example is listed below.

- `test_subprocess_ex4.py`:

```python
import os
from time import sleep
from multiprocessing import Process
from multiprocessing import Value


# custom process class
class CustomProcess(Process):
  # override the constructor
  def __init__(self, name: str):
    # execute the base constructor
    Process.__init__(self)
    self.name = name

    # initialize integer attribute
    self.data = Value('i', 0)

  @property
  def identity(self):
    return f'{self.name}:{os.getpid()}'

  # override the run function
  def run(self):
    # block for a moment
    sleep(1)
    # store the data variable
    self.data.value = 99
    # report stored value
    print(f'{self.identity} > Child stored: {self.data.value}')


# entry point
if __name__ == '__main__':
  print(f'Main process with pid: {os.getpid()}')

  # create the process
  process = CustomProcess('John')
  print(f'Parent with process.data.value: {process.data.value}')

  # start the process
  process.start()

  # wait for the process to finish
  print('Waiting for the child process to finish')
  # block until child process is terminated
  process.join()

  # report the process attribute
  print(f'Parent got: {process.data.value}')
```

Execution result:
```shell
$ ./test_subprocess_ex4.py 
Main process with pid: 850814
Parent with process.data.value: 0
Waiting for the child process to finish
John:850815 > Child stored: 99
Parent got: 99
```

You can learn more about returning values from a child process in this tutorial "[**Shared Process Class Attributes in Python**](https://superfastpython.com/share-process-attributes/)". Next, let’s take a closer look at process start methods.

<a id='sect4'></a>
## <b><font color='darkblue'>Process Start Methods</font></b> ([back](#agenda))
<b><font size='3ptx'>In multiprocessing, we may need to change the technique used to start child processes.</font></b>
* <b><a href='#sect4_1'>What is a Start Method</a></b>
* <b><a href='#sect4_2'>How to Change The Start Method</a></b>
* <b><a href='#sect4_3'>How to Set Start Method Via Context</a></b>
* <b><a href='#sect4_4'>Process Instance Attributes</a></b>


This is called the start method.

<a id='sect4_1'></a>
### <b><font color='darkgreen'>What is a Start Method</font></b>
<b><font size='3ptx'>A start method is the technique used to start child processes in Python.</font></b>

There are three start methods, they are:
* **spawn**: start a new Python process.
* **fork**: copy a Python process from an existing process.
* **forkserver**: new process from which future forked processes will be copied.

Each platform has a default start method. The following lists the major platforms and the default start methods:
* **Windows** (win32): spawn
* **macOS** (darwin): spawn
* **Linux** (unix): fork

Not all platforms support all start methods. The following lists the major platforms and the start methods that are supported:
* **Windows** (win32): spawn
* **macOS** (darwin): spawn, fork, forkserver.
* **Linux** (unix): spawn, fork, forkserver.

<a id='sect4_2'></a>
### <b><font color='darkgreen'>How to Change The Start Method</font></b> ([back](#sect4))
<b><font size='3ptx'>The [multiprocessing](https://docs.python.org/3/library/multiprocessing.html#) module provides functions for getting and setting the start method for creating child processes.</font></b>

The list of supported start methods can be retrieved via the [**multiprocessing**.get_all_start_methods()](get_all_start_methods) function. The function returns a list of string values, each representing a supported start method.

In [3]:
# get supported start methods
methods = multiprocessing.get_all_start_methods()
print(methods)

['fork', 'spawn', 'forkserver']


The current start method can be retrieved via the [**multiprocessing**.get_start_method()](https://docs.python.org/3/library/multiprocessing.html#multiprocessing.get_start_method) function. The function returns a string value for the currently configured start method.

In [4]:
# get the current start method
method = multiprocessing.get_start_method()
print(method)

fork


The start method can be set via the [**multiprocessing**.set_start_method()](https://docs.python.org/3/library/multiprocessing.html#multiprocessing.set_start_method) function. For example:

```python
...
# set the start method
multiprocessing.set_start_method('spawn')
```

It is a best practice, and required on most platforms that the start method be set first, prior to any other code, and to be done so within a `if __name__ == ‘__main__’` check called a [protected entry point or top-level code environment](https://docs.python.org/3/library/__main__.html).

For example:
```python
# protect the entry point
if __name__ == '__main__':
	# set the start method
	multiprocessing.set_start_method('spawn')
```

In summary, the rules for setting the start method are as follows:
* Set the start method first prior to all other code.
* Set the start method only once in a program.
* Set the start method within a protected entry point.

<a id='sect4_3'></a>
### <b><font color='darkgreen'>How to Set Start Method Via Context</font></b> ([back](#sect4))
<b><font size='3ptx'>A multiprocessing context configured with a given start method can be retrieved via the [multiprocessing.get_context()](https://docs.python.org/3/library/multiprocessing.html#multiprocessing.get_context) function.</font></b>

This function takes the name of the start method as an argument, then returns a multiprocessing context that can be used to create new child processes.

For example:
```python
...
# get a context configured with a start method
context = multiprocessing.get_context('fork')
```

The context can then be used to create a child process, for example:
```python
...
# create a child process via a context
process = context.Process(...)
```

It may also be possible to force the start method. This can be achieved via the “force” argument provided on the [set_start_method()](https://docs.python.org/3/library/multiprocessing.html#multiprocessing.set_start_method) implementation in the DefaultContext, although not documented.

For exampl
```python
...
# set the start method
context.set_start_method('spawn', force=True)
```e:

You can learn more about setting the process start method in the tutorial "[**Multiprocessing Start Methods**](https://superfastpython.com/multiprocessing-start-method)". Next, let’s look at some properties of processes instances.

<a id='sect5'></a>
## <b><font color='darkblue'>Process Instance Attributes</font></b> ([back](#agenda))
<b><font size='3ptx'>An instance of the [multiprocessing.Process](https://docs.python.org/3/library/multiprocessing.html#the-process-class) class provides a handle of a new instance of the Python interpreter.</font></b>
* <b><a href='#sect5_1'>Query Process Name</a></b>
* <b><a href='#sect5_2'>Query Process Daemon</a></b>
* <b><a href='#sect5_3'>Query Process PID</a></b>
* <b><a href='#sect5_4'>Query Process Alive</a></b>
* <b><a href='#sect5_5'>Query Process Exit Code </a></b>

As such, it provides attributes that we can use to query properties and the status of the underlying process. Let’s look at some examples.

<a id='sect5_1'></a>
### <b><font color='darkgreen'>Query Process Name</font></b>
<b><font size='3ptx'>Each process has a name.</font></b>

The parent process has the name “`MainProcess`“. Child processes are named automatically in a somewhat unique manner within each process with the form “`Process-%d`” where %d is the integer indicating the process number created by the parent process, e.g. Process-1 for the first process created.

We can access the name of a process via the [**multiprocessing.Process**.name](https://docs.python.org/3/library/multiprocessing.html#multiprocessing.Process.name) attribute, for example:

```python
...
# report the process name
print(process.name)
```

The example below creates an instance of the [**multiprocessing.Process**](https://docs.python.org/3/library/multiprocessing.html#the-process-class) class and reports the default name of the process:
```python
# SuperFastPython.com
# example of accessing the child process name
from multiprocessing import Process
# entry point
if __name__ == '__main__':
    # create the process
    process = Process()
    # report the process name
    print(process.name)
```

You can learn more about configuring the process name in the tutorial "[**How to Change the Process Name in Python**](https://superfastpython.com/process-name/)".

<a id='sect5_2'></a>
### <b><font color='darkgreen'>Query Process Daemon</font></b> ([back](#sect))
Daemon process is the name given to the background process. By default, processes are non-daemon processes because they inherit the daemon value from the parent process, which is set `False` for the MainProcess.

<b>A Python parent process will only exit when all non-daemon processes have finished exiting</b>. For example, the MainProcess is a non-daemon process. This means that the daemon process can run in the background and do not have to finish or be explicitly excited for the program to end.

We can determine if a process is a daemon process via the [**multiprocessing.Process**.daemon](https://docs.python.org/3/library/multiprocessing.html#multiprocessing.Process.daemon) attribute.

<a id='sect5_3'></a>
### <b><font color='darkgreen'>Query Process PID</font></b> ([back](#sect5))
<font size='3ptx'><b>Each process has a unique process identifier, called the PID, assigned by the operating system.</b></font>

Python processes are real native processes, meaning that each process we create is actually created and managed by the underlying operating system. As such, the operating system will assign a unique integer to each process that is created on the system<font color='brown'> (across process</font>es).

The process identifier can be accessed via [**the multiprocessing.Pro**cess](https://docs.python.org/3/library/multiprocessing.html#multiprocessing.Process.pid).pid property and is assigned after the process has been started.

<a id='sect5_4'></a>
### <b><font color='darkgreen'>Query Process Alive</font></b> ([back](#sect5))
<b><font size='3ptx'>A process instance can be alive or dead.</font></b>

An alive process means that the <font color='blue'>run()</font> method of the [**multiprocessing.Process**](https://docs.python.org/3/library/multiprocessing.html#the-process-class) instance is currently executing.

This means that before the <font color='blue'>start()</font> method is called and after the <font color='blue'>run()</font> method has completed, the process will not be alive.

We can check if a process is alive via the [**multiprocessing.Process**.is_alive()](https://docs.python.org/3/library/multiprocessing.html#multiprocessing.Process.is_alive) method.

<a id='sect5_5'></a>
### <b><font color='darkgreen'>Query Process Exit Code</font></b> ([back](#sect5))
<b><font size='3ptx'>A child process will have an exit code once it has terminated.</font></b>

An exit code provides an indication of whether processes completed successfully or not, and if not, the type of error that occurred that caused the termination.

Common exit codes include:
* 0: Normal (exit success)
* 1: Error (exit failure)

We can write the exit code for a child process via the [**multiprocessing.Process**.exitcode](https://docs.python.org/3/library/multiprocessing.html#multiprocessing.Process.exitcode) attribute.

<a id='sect6'></a>
## <b><font color='darkblue'>Configure Processes</font></b> ([back](#agenda))
<b><font size='3ptx'>Instances of the [**multiprocessing.Process**](https://docs.python.org/3/library/multiprocessing.html#the-process-class) class can be configured.</font></b>
* <b><a href='#sect6_1'>How to Configure Process Name</a></b>
* <b><a href='#sect6_2'>How to Configure a Daemon Process</a></b>

There are two properties of a process that can be configured, they are the name of the process and whether the process is a daemon or not.

<a id='sect6_1'></a>
### <b><font color='darkgreen'>How to Configure Process Name</font></b>
<b>Processes can be assigned custom names.</b>

The name of a process can be set via the “name” argument in the [**multiprocessing.Process**](https://docs.python.org/3/library/multiprocessing.html#the-process-class) constructor. e.g.:
```python
...
# create a process with a custom name
process = Process(name='MyProcess')
```

<a id='sect6_2'></a>
### <b><font color='darkgreen'>How to Configure a Daemon Process</font></b>
<b>Processes can be configured to be “daemon” or “daemonic”, that is, they can be configured as background processes.</b>

A parent Python process can only exit once all non-daemon child processes have exited.

This means that daemon child processes can run in the background and do not prevent a Python program from exiting when the “main parts of the program” have finished.

A process can be configured to be a daemon by setting the “daemon” argument to True in the [**multiprocessing.Process**](https://docs.python.org/3/library/multiprocessing.html#the-process-class) constructor.

For example:
```python
# create a daemon process
process = Process(daemon=True)
```

<a id='sect7'></a>
## <b><font color='darkblue'>Main Process</font></b> ([back](#agenda))
<b><font size='3ptx'>The main process is the parent process that executes your program.</font></b>
* <b><a href='#sect7_1'>What is the Main Process?</a></b>
* <b><a href='#sect7_2'>How Can the Main Process Be Identified</a></b>
* <b><a href='#sect7_3'>How to Get the Main Process?</a></b>

In this section we will take a closer look at the main process in Python.

<a id='sect7_1'></a>
### <b><font color='darkgreen'>What is the Main Process?</font></b>
<b><font size='3ptx'>The main process in Python is the process started when you run your Python program.</font></b>

<b>Recall that a process in Python is an instance of the Python interpreter</b>. We can create and run new child processes via the [**multiprocessing.Process**](https://docs.python.org/3/library/multiprocessing.html#the-process-class) class.

What makes the main process unique is that it is the first process created when you run your program and the main thread of the process executes the entry point of your program.

As such the main process does not have a parent process.

The main process also has a distinct name, specific`ally “MainP`rocess“.

<a id='sect7_2'></a>
### <b><font color='darkgreen'>How Can the Main Process Be Identified</font></b>
There are a number of ways that the main process can be identified. They are:
* The main process has a distinct name of “MainProcess“.
* The main process does not have a parent process.
* The main process is an instance of t[**he multiprocessing.process._MainProc**](https://github.com/python/cpython/blob/3.10/Lib/multiprocessing/process.py#L391)ess class.

<a id='sect7_3'></a>
### <b><font color='darkgreen'>How to Get the Main Process?</font></b> ([back](#sect7))
<b><font size='3ptx'>We can get a [multiprocessing.Process](https://docs.python.org/3/library/multiprocessing.html#the-process-class) instance for the main process.</font></b>

There are a number of ways that we can do this, depending on where exactly we are in the code.

For example, if we are running code in the main process, we can get a [**multiprocessing.Process**](https://docs.python.org/3/library/multiprocessing.html#the-process-class) instance for the main process via the [**multiprocessing**.current_process()](https://docs.python.org/3/library/multiprocessing.html#multiprocessing.current_process) function. For example:
```python
# SuperFastPython.com
# example of getting the main process from the main process
from multiprocessing import current_process
# get the process instance
process = current_process()
# report details of the main process
print(process)
```

If we are in a child process of the main process then we can get the main process via the [**multiprocessing**.parent_process()](https://docs.python.org/3/library/multiprocessing.html#multiprocessing.parent_process)  function. For example:
```python
# SuperFastPython.com
# example of getting the main process from a child of the main process
from multiprocessing import parent_process
from multiprocessing import Process
 
# function executed in a child process
def task():
    # get the parent process instance
    process = parent_process()
    # report details of the main process
    print(process)
 
# entry point
if __name__ == '__main__':
    # create a new process
    process = Process(target=task)
    # start the new process
    process.start()
    # wait for the new process to terminate
    process.join()
```

<a id='sect8'></a>
## <b><font color='darkblue'>Process Utilities</font></b> ([back](#agenda))
<b><font size='3ptx'>There are a number of utilities we can use when working with processes within a Python process.</font></b>
* <b><a href='#sect8_1'>Active Child Processes</a></b>
* <b><a href='#sect8_2'>Get The Number of CPU Cores</a></b>


In this section we will review a number of additional utility functions.

<a id='sect8_1'></a>
### <b><font color='darkgreen'>Active Child Processes</font></b>
<b><font size='3ptx'>We can get a list of all active child processes for a parent process.</font></b>

This can be achieved via th[**e multiprocessi**ng.active_children](https://docs.python.org/3/library/multiprocessing.html#multiprocessing.active_children)() function that returns a list of all child processes that are currently running. For example:
```python
# SuperFastPython.com
# list all active child processes
from time import sleep
from multiprocessing import active_children
from multiprocessing import Process
 
# function to execute in a new process
def task():
    # block for a moment
    sleep(1)
 
# entry point
if __name__ == '__main__':
    # create a number of child processes
    processes = [Process(target=task) for _ in range(5)]
    # start the child processes
    for process in processes:
        process.start()
    # get a list of all active child processes
    children = active_children()
    # report a count of active children
    print(f'Active Children Count: {len(children)}')
    # report each in turn
    for child in children:
        print(child)
```

<a id='sect8_2'></a>
### <b><font color='darkgreen'>Get The Number of CPU Cores</font></b> ([back](#sect8))
<b><font size='3ptx'>We may want to know the number of CPU cores available.</font></b>

This can be determined via th[**e multiprocessi**ng.cpu_count](https://docs.python.org/3/library/multiprocessing.html#multiprocessing.cpu_count)() function.

The function returns an integer that indicates the number of logical CPU cores available in the system running the Python program.

Recall that a central processing unit or CPU executes program instructions. A CPU may have one or more physical CPU cores for executing code in parallel. Modern CPU cores may make u[**se of hyperthr**](https://en.wikipedia.org/wiki/Hyper-threading)eading, allowing each physical CPU core to operate like two (or more) logical CPU cores.

Therefore, a computer system with a CPU with four physical cores may report eight logical CPU cores [**multiprocessing**.cpu_count()](https://docs.python.org/3/library/multiprocessing.html#multiprocessing.cpu_count)_count() function function.

It may be useful to know the number of CPU cores available to configure a thread pool or the number of processes to create to execute tasks in parallel.

The example below demonstrates how to use the number of CPU cores.

In [5]:
print(f'This machine has {multiprocessing.cpu_count()} CPU core(s)')

This machine has 24 CPU core(s)


In [6]:
!cat /proc/cpuinfo | grep 'process' | wc -l

24


In [7]:
!lscpu | grep '^CPU(s):'

CPU(s):                             24


<a id='sect9'></a>
## <b><font color='darkblue'>Process Mutex Lock</font></b> ([back](#agenda))
<b><font size='3ptx'>A mutex lock is used to protect critical sections of code from concurrent execution.</font></b>
* <b><a href='#sect9_1'>What is a Mutual Exclusion Lock</a></b>
* <b><a href='#sect9_2'>How to Use a Mutex Lock</a></b>
* <b><a href='#sect9_3'>Example of Using a Mutex Lock</a></b>

You can use a mutual exclusion (mutex) lock in Python via the [**multiprocessing.Lock**](https://docs.python.org/3/library/multiprocessing.html#multiprocessing.Lock) class.

<a id='sect9_1'></a>
### <b><font color='darkgreen'>What is a Mutual Exclusion Lock</font></b>
<b><font size='3ptx'>A [mutual exclusion lock](https://en.wikipedia.org/wiki/Mutual_exclusion) or mutex lock is a synchronization primitive intended to prevent a race condition.</font></b>

A race condition is a concurrency failure case when two processes (or threads) run the same code and access or update the same resource (e.g. data variables, stream, etc.) leaving the resource in an unknown and inconsistent state.

<b>Race conditions often result in unexpected behavior of a program and/or corrupt data.</b>

These sensitive parts of code that can be executed by multiple processes concurrently and may result in race conditions are called critical sections. A critical section may refer to a single block of code, but it also refers to multiple accesses to the same data variable or resource from multiple functions.

A mutex lock can be used to ensure that only one process at a time executes a critical section of code at a time, while all other processes trying to execute the same code must wait until the currently executing process is finished with the critical section and releases the lock.

Each process must attempt to acquire the lock at the beginning of the critical section. If the lock has not been obtained, then a process will acquire it and other processes must wait until the process that acquired the lock releases it.

If the lock has not been acquired, we might refer to it as being in the “unlocked” state. Whereas if the lock has been acquired, we might refer to it as being in the “locked” state.

* **Unlocked**: The lock has not been acquired and can be acquired by the next process that makes an attempt.
* **Locked**: The lock has been acquired by process thread and any process that makes an attempt to acquire it must wait until it is released.

Locks are created in the unlocked state.

Now that we know what a mutex lock is, let’s take a look at how we can use it in Python.

<a id='sect9_2'></a>
### <b><font color='darkgreen'>How to Use a Mutex Lock</font></b>
<b><font size='3ptx'>Python provides a mutual exclusion lock for use with processes via the [multiprocessing.Lock](https://docs.python.org/3/library/multiprocessing.html#multiprocessing.Lock) class.</font></b>

An instance of the lock can be created and then acquired by processes before accessing a critical section, and released after the critical section. For exampl
```python
...
# create a lock
lock = multiprocessing.Lock()
# acquire the lock
lock.acquire()
# ...
# release the lock
lock.release()
```e:

<b>Only one process can have the lock at any time. If a process does not release an acquired lock, it cannot be acquired again</b>.

The process attempting to acquire the lock will block until the lock is acquired, such as if another process currently holds the lock then releases it.

We can attempt to acquire the lock without blocking by setting the “block” argument to False. <b>If the lock cannot be acquired, a value of `False` is returned.</b>
```python
...
# acquire the lock without blocking
lock.acquire(block=false)
```

We can also attempt to acquire the lock with a timeout, that will wait the set number of seconds to acquire the lock before giving up. If the lock cannot be acquired, a value of False is returned.
```python
...
# acquire the lock with a timeout
lock.acquire(timeout=10)
```

We can also use the lock via the context manager protocol via the with statement, allowing the critical section to be a block within the usage of the lock and for the lock to be released automatically once the block has completed.

For example:
```python
...
# create a lock
lock = multiprocessing.Lock()
# acquire the lock
with lock:
    # ...
```

This is the preferred usage as it makes it clear where the protected code begins and ends, and ensures that the lock is always released, even if there is an exception or error within the critical section.

We can also check if the lock is currently acquired by a process via th<font color='blue'>e locked</font>() function.
```python
...
# check if a lock is currently acquired
if lock.locked():
    # ...
```

Now that we know how t[**o use the multiproce**](https://docs.python.org/3/library/multiprocessing.html#multiprocessing.Lock)ssing.Lock class, let’s look at a worked example.

<a id='sect9_3'></a>
### <b><font color='darkgreen'>Example of Using a Mutex Lock</font></b> ([back](#sect9))
<b><font size='3ptx'>We can develop an example to demonstrate how to use the mutex lock.</font></b>

The complete example of using a lock is listed below:
- `test_subprocess_ex5.py`:

```python
# example of a mutual exclusion (mutex) lock for processes
from datetime import datetime
from time import sleep
from random import random
from multiprocessing import Process
from multiprocessing import Lock


# work function
def task_with_lock(lock, identifier, value):
  # acquire the lock
  with lock:
    print(f'>process {identifier} got the lock, sleeping for {value:.03f}s')
    sleep(value)


def task_ignore_lock(lock, identifier, value):
  # acquire the lock
  print(f'>process {identifier} got the lock, sleeping for {value:.03f}s')
  sleep(value)


# entry point
if __name__ == '__main__':
  # create the shared lock
  lock = Lock()
  # create a number of processes with different sleep times
  random_numbers = [random() for i in range(10)]
  random_number_sum = sum(random_numbers)
  processes = [Process(target=task_ignore_lock, args=(lock, i, random_numbers[i])) for i in range(10)]
  # start the processes
  for process in processes:
    process.start()

  # wait for all processes to finish
  start_time = datetime.now()
  for process in processes:
    process.join()

  time_diff = datetime.now() - start_time
  print(f'Done in {time_diff}  (random nubmer sum={random_number_sum:.03f})!')
`````

You can learn more about mutex locks in this tutorial "[**Multiprocessing Lock in Python**](https://superfastpython.com/multiprocessing-mutex-lock-in-python/)". Now that we are familiar with multiprocessing mutex locks, let’s take a look at the reentrant lock.

<a id='sect10'></a>
## <b><font color='darkblue'>Process Reentrant Lock</font></b> ([back](#agenda))
<b><font size='3ptx'>A reentrant lock is a lock that can be acquired more than once by the same process.</font></b>
* <b><a href='#sect10_1'>What is a Reentrant Lock</a></b>
* <b><a href='#sect10_2'>How to Use the Reentrant Lock</a></b>
* <b><a href='#sect10_3'>Example of Using a Reentrant Lock</a></b>

You can use reentrant locks in Python via the [**multiprocessing.RLock**](https://docs.python.org/3/library/multiprocessing.html#multiprocessing.RLock) class.

<a id='sect10_1'></a>
### <b><font color='darkgreen'>What is a Reentrant Lock</font></b>
<b><font size='3ptx'>A reentrant mutual exclusion lock, “reentrant mutex” or “reentrant lock” for short, is like a mutex lock except it allows a process (or thread) to acquire the lock more than once</font>.</b>

<b>A process may need to acquire the same lock more than once for many reasons.</b>

We can imagine critical sections spread across a number of functions, each protected by the same lock. A process may call across these functions in the course of normal execution and may call into one critical section from another critical section.

<b>A limitation of a</b> (<font color='brown'>non-reentrant</font>) <b>mutex lock is that if a process has acquired the lock that it cannot acquire it again. In fact, this situation will result in a [deadlock](https://en.wikipedia.org/wiki/Deadlock) as it will wait forever for the lock to be released so that it can be acquired, but it holds the lock and will not release it.</b>

<b>A reentrant lock will allow a process to acquire the same lock again if it has already acquired it</b>. This allows the process to execute critical sections from within critical sections, as long as they are protected by the same reentrant lock.

Each time a process acquires the lock it must also release it, meaning that there are recursive levels of acquire and release for the owning process. As such, this type of lock is sometimes called a “recursive mutex lock“.

Now that we are familiar with the reentrant lock, let’s take a closer look at the difference between a lock and a reentrant lock in Python.

<a id='sect10_2'></a>
### <b><font color='darkgreen'>How to Use the Reentrant Lock</font></b> ([back](#sect10))
<b><font size='3ptx'>Python provides a reentrant lock for processes via the [multiprocessing.RLock](https://docs.python.org/3/library/multiprocessing.html#multiprocessing.RLock) class.</font></b>

An instance of the [**multiprocessing.RLock**](https://docs.python.org/3/library/multiprocessing.html#multiprocessing.RLock) can be created and then acquired by processes before accessing a critical section, and released after the critical section:
```python
...
# create a reentrant lock
lock = multiprocessing.RLock()
# acquire the lock
lock.acquire()
# ...
# release the lock
lock.release()
```

We can attempt to acquire the lock without blocking by setting the “block” argument to False. If the lock cannot be acquired, a value of `False` is returned:
```python
...
# acquire the lock without blocking
lock.acquire(block=false)
```

We can also attempt to acquire the lock with a timeout, that will wait the set number of seconds to acquire the lock before giving up. If the lock cannot be acquired, a value of `False` is returned.
```python
...
# acquire the lock with a timeout
lock.acquire(timeout=10)
```

We can also use the reentrant lock via the context manager protocol via the `with` statement, allowing the critical section to be a block within the usage of the lock and for the lock to be released once the block is exited. For example:
```python
...
# create a reentrant lock
lock = multiprocessing.RLock()
# acquire the lock
with lock:
    # ...
```

Now that we know how to use the [**multiprocessing.RLock**](https://docs.python.org/3/library/multiprocessing.html#multiprocessing.RLock) class, let’s look at a worked example.

<a id='sect10_3'></a>
### <b><font color='darkgreen'>Example of Using a Reentrant Lock</font></b> ([back](#sect10))
The complete example of using a lock is listed below.
- `test_subprocess_ex6.py`

```python
# example of a reentrant lock for processes
from time import sleep
from random import random
from multiprocessing import Process
from multiprocessing import RLock


# reporting function
def report(lock, identifier):
  # acquire the lock
  with lock:
    print(f'>process {identifier} done')


# work function
def task(lock, identifier, value):
  # acquire the lock
  with lock:
    print(f'>process {identifier} sleeping for {value}')
    sleep(value)
    # report
    rep


# entry point
if __name__ == '__main__':
  # create a shared reentrant lock
  lock = RLock()

  # create processes
  processes = [Process(target=task, args=(lock, i, random())) for i in range(10)]

  # start child processes
  for process in processes:
    process.start()

  # wait for child processes to finish
  for process in processes:
    process.join()ort(lock, identifier)
```

You can learn more about reentrant locks in this tutorial "[**Multiprocessing RLock in Python**](https://superfastpython.com/multiprocessing-rlock-in-python/)". Now that we are familiar with the reentrant lock, let’s look at condition variables.

## <b><font color='darkblue'>Supplement</font></b>
* [Python Concurrency - What’s are the Differences between Processes and Threads](https://www.pythontutorial.net/python-concurrency/differences-between-processes-and-threads/)
* [Generate CPU load using Python](https://qxf2.com/blog/generate-cpu-load/)